In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import warnings


%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
churn = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')

In [ ]:
churn.head()

In [ ]:
#churn[cat_col].head()
df = churn
# Since surname, consumerid and rownumber does not have any impact w.r.t Exited, dropping.
df = churn.drop(['RowNumber','CustomerId','Surname'], axis = 1)

In [ ]:
# Checking missing values 
#churn.info()
def num_cat_features(df):
    num_cols = []
    cat_cols = []
    s= df.dtypes
    for i in s.index:
        if (s[i] == 'int64' or s[i] == 'float64'):
            num_cols.append(i)
        elif s[i]== 'object':
            cat_cols.append(i)
        else:
            pass
    return num_cols, cat_cols

num_col, cat_col = num_cat_features(df)

In [ ]:
df[num_col].head()
df[cat_col].head()

In [ ]:
# Dummies in Geography and Gender
df_cat = df[cat_col]
geo = pd.get_dummies(df_cat['Geography'], prefix = 'Geo')
gender = pd.get_dummies(df_cat['Gender'])
df.drop(cat_col,axis =1, inplace = True)
df = pd.concat([geo,gender,df], axis =1 )
df.head()

In [ ]:
#Scaling numeric columns
cols_scale = ['CreditScore','Tenure','Balance','NumOfProducts', 'EstimatedSalary']

In [ ]:
plt.figure(figsize = (15,15))
sns.heatmap(df.corr(), annot = True)

In [ ]:
sns.set(style = "darkgrid")
fig = plt.figure(figsize = (20,20))
ax1 = fig.add_subplot(3,3,1)
ax1. set_title("M vs F")

ax2 = fig.add_subplot(3,3,2)
ax2. set_title("M vs F - Exited")

sns.barplot(x = churn['Gender'].value_counts().sort_index().index,y = churn['Gender'].value_counts().sort_index().values, ax= ax1 )
sns.countplot(x = 'Exited', hue = 'Gender', data = churn, ax = ax2)


More females exit than males. 

In [ ]:
fig = plt.figure(figsize = (20,18))
ax3 = fig.add_subplot(3,3,1)
ax3.set_title("Geos")

ax4 = fig.add_subplot(3,3,2)
ax4.set_title("Geos - Exited")

sns.barplot(x = churn['Geography'].value_counts().sort_index().index,y = churn['Geography'].value_counts().sort_index().values, ax = ax3)
sns.countplot(x = 'Exited', hue = 'Geography', data = churn, ax = ax4)

In [ ]:
fig = plt.figure(figsize = (20,18))
ax5 = fig.add_subplot(3,3,1)
ax5.set_title("Product")

ax6 = fig.add_subplot(3,3,2)
ax6.set_title("Product - Exited")

sns.barplot(x = churn['NumOfProducts'].value_counts().sort_index().index,y = churn['NumOfProducts'].value_counts().sort_index().values, ax = ax5)
sns.countplot(x = 'Exited', hue = 'NumOfProducts', data = churn, ax = ax6)

In [ ]:
churn.head()

In [ ]:
plt.figure(figsize = (10,7))
sns.distplot(churn['Age'], kde = False ,bins = 10)
plt.ylabel("Counts")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier



In [ ]:
df.head()

In [ ]:
X = df.drop('Exited', axis =1)
y = df['Exited']

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12)

**Logistic Regression**

In [ ]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
# Accuracy on Training set
from sklearn.model_selection import cross_val_predict
y_prob_train_lr = cross_val_predict(log_reg, X_train, y_train, cv = 10, method = 'decision_function')

# y_pred_train_lr = log_reg.predict(X_train)
# y_prob_train_lr = log_reg.predict_proba(X_train)

from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score ,confusion_matrix
fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_lr)
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_lr)

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - Logistic Regression - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall - Logistic Regression - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - Logistic Regression - Training")


lr_auc_score_train = roc_auc_score(y_train,y_prob_train_lr ) # 0.76
# lr_auc_score_train



**Decision Tree Classifier**

In [ ]:
# Decision Tree Classifier
dtree_clf = DecisionTreeClassifier()
dtree_clf.fit(X_train, y_train)
# Accuracy on Training set
from sklearn.model_selection import cross_val_predict
y_prob_train_dtree = cross_val_predict(dtree_clf, X_train, y_train, cv = 10, method = 'predict_proba')

fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_dtree[:,-1])
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_dtree[:,-1])

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - Decision Tree Classifier - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall -  Decision Tree Classifier - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - Decision Tree Classifier- Training")


dtree_auc_score_train = roc_auc_score(y_train,y_prob_train_dtree[:,-1] ) # 0.67
# dtree_auc_score_train 

In [ ]:
# Linear SVC  Classifier
svc_clf = LinearSVC(C = 1.0,random_state = 12, max_iter = 1500, loss = 'squared_hinge')
svc_clf.fit(X_train, y_train)
# Accuracy on Training set
y_prob_train_svc = cross_val_predict(svc_clf, X_train, y_train, cv = 10, method = 'decision_function')

fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_svc)
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_svc)

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - Linear SVC Classifier - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall -  Linear SVC Classifier - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - Linear SVC Classifier- Training")


svc_auc_score_train = roc_auc_score(y_train,y_prob_train_svc) # 0.77
# svc_auc_score_train 

In [ ]:
# SVM  Classifier
from sklearn.svm import SVC
svm_clf = SVC(random_state = 12)
svm_clf.fit(X_train, y_train)
# Accuracy on Training set
y_prob_train_svm = cross_val_predict(svm_clf, X_train, y_train, cv = 10, method = 'decision_function')

fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_svm)
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_svm)

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - SVM Classifier - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall -  SVM Classifier - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - SVM Classifier- Training")


svm_auc_score_train = roc_auc_score(y_train,y_prob_train_svm) # 0.82
# svm_auc_score_train 

In [ ]:
# Random Forest  Classifier
from sklearn.ensemble import RandomForestClassifier
rdm_frst_clf = RandomForestClassifier(random_state = 12, max_depth = 7, n_estimators=20)
rdm_frst_clf.fit(X_train, y_train)
# Accuracy on Training set
y_prob_train_forst = cross_val_predict(rdm_frst_clf, X_train, y_train, cv = 10, method = 'predict_proba')

fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_forst[:,-1])
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_forst[:,-1])

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - Random Forest Classifier - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall -  Random Forest Classifier - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - Random Forest Classifier - Training")


frst_auc_score_train = roc_auc_score(y_train,y_prob_train_forst[:,-1]) # 0.85
# frst_auc_score_train 

In [ ]:
# Stochastic Gradient Descent  Classifier
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(alpha = 1,random_state = 12, penalty = 'l2', loss = 'modified_huber')
sgd_clf.fit(X_train, y_train)
# Accuracy on Training set
y_prob_train_sgd = cross_val_predict(sgd_clf, X_train, y_train, cv = 10, method = 'decision_function')

fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_sgd)
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_sgd)

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - SGD Classifier - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall -  SGD Classifier - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - SGD Classifier - Training")


sgd_auc_score_train = roc_auc_score(y_train,y_prob_train_sgd) # 0.77
# sgd_auc_score_train 

In [ ]:
# KNN  Classifier
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='distance')
knn_clf.fit(X_train, y_train)
# Accuracy on Training set
y_prob_train_knn = cross_val_predict(knn_clf, X_train, y_train, cv = 10, method = 'predict_proba')

fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_knn[:,-1])
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_knn[:,-1])

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - KNN Classifier - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall -  KNN Classifier - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - KNN Classifier - Training")


knn_auc_score_train = roc_auc_score(y_train,y_prob_train_knn[:,-1]) # 0.78
# sgd_auc_score_train 

In [ ]:
# XGboost  Classifier
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(random_state = 12)
xgb_clf.fit(X_train, y_train)
# Accuracy on Training set
y_prob_train_xgb = cross_val_predict(xgb_clf, X_train, y_train, cv = 10, method = 'predict_proba')

fpr, tpr, thrshld = roc_curve(y_train, y_prob_train_xgb[:,-1])
precision, recall, thrshld = precision_recall_curve(y_train, y_prob_train_xgb[:,-1])

fig = plt.figure(figsize = (70,50))
# Precision / Recall vs Threshold
plt.subplot(10,10,1)
plt.plot(thrshld, precision[:-1], "b-", label="Precision")
plt.plot(thrshld, recall[:-1], "r-", label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision / Recall")
plt.title("Precision / Recall vs Threshold - XGB Classifier - Training")
plt.legend()


# Precision Vs Recall
plt.subplot(10,10,2)
plt.plot(recall, precision, 'b-')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision vs Recall -  XGB Classifier - Training")


# ROC Curve
plt.subplot(10,10,3)
plt.plot([[0,0], [1,1]], 'k--')
plt.plot(fpr, tpr, 'g-')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve - XGB Classifier - Training")


xgb_auc_score_train = roc_auc_score(y_train,y_prob_train_xgb[:,-1]) # 0.86
# xgb_auc_score_train 

In [ ]:
# Performance on Test data
y_pred_log = log_reg.predict(X_test)
cm_log = confusion_matrix(y_test, y_pred_log)

y_pred_dtree = dtree_clf.predict(X_test)
cm_dtree = confusion_matrix(y_test, y_pred_dtree)

y_pred_lsvc = svc_clf.predict(X_test)
cm_lsvc = confusion_matrix(y_test, y_pred_lsvc)

y_pred_svm = svm_clf.predict(X_test)
cm_svm = confusion_matrix(y_test, y_pred_svm)

y_pred_frst = rdm_frst_clf.predict(X_test)
cm_frst = confusion_matrix(y_test, y_pred_frst)

y_pred_sgd = sgd_clf.predict(X_test)
cm_sgd = confusion_matrix(y_test, y_pred_sgd)

y_pred_knn = knn_clf.predict(X_test)
cm_knn = confusion_matrix(y_test, y_pred_knn)

y_pred_xgb = xgb_clf.predict(X_test)
cm_xgb = confusion_matrix(y_test, y_pred_xgb)

In [ ]:
fig = plt.figure(figsize = (15,15))

ax1 = fig.add_subplot(3,3,1)
ax1.set_title("Logistic Regression")

ax2 = fig.add_subplot(3,3,2)
ax2.set_title("Decision Tree Classifier")

ax3 = fig.add_subplot(3,3,3)
ax3.set_title("Linear SVC Classifier")

ax4 = fig.add_subplot(3,3,4)
ax4.set_title("SVM Classifier")

ax5 = fig.add_subplot(3,3,5)
ax5.set_title("Random Forest Classifier")

ax6 = fig.add_subplot(3,3,6)
ax6.set_title("SGD Classifier")

ax7 = fig.add_subplot(3,3,7)
ax7.set_title("KNN Classifier")

ax8 = fig.add_subplot(3,3,8)
ax8.set_title("XGB Classifier")

sns.heatmap(cm_log, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax1)
sns.heatmap(cm_dtree, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax2)
sns.heatmap(cm_lsvc, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax3)
sns.heatmap(cm_svm, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax4)
sns.heatmap(cm_frst, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax5)
sns.heatmap(cm_sgd, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax6)
sns.heatmap(cm_knn, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax7)
sns.heatmap(cm_xgb, annot = True, cmap = 'RdBu', fmt = 'd', ax = ax8)


In [ ]:
# Testing Scores
log_acc =   (cm_log[0,0] + cm_log[1,1]) / cm_log.sum()
dtree_acc = (cm_dtree[0,0] + cm_dtree[1,1]) / cm_dtree.sum()
lsvc_acc = (cm_lsvc[0,0] + cm_lsvc[1,1]) / cm_lsvc.sum()
svm_acc = (cm_svm[0,0] + cm_svm[1,1]) / cm_svm.sum()
frst_acc = (cm_frst[0,0] + cm_frst[1,1]) / cm_frst.sum()
sgd_acc = (cm_sgd[0,0] + cm_sgd[1,1]) / cm_sgd.sum()
knn_acc = (cm_knn[0,0] + cm_knn[1,1]) / cm_knn.sum()
xgb_acc = (cm_xgb[0,0] + cm_xgb[1,1]) / cm_xgb.sum()


In [ ]:
acc_list = [log_acc,dtree_acc,lsvc_acc,svm_acc,frst_acc,sgd_acc,knn_acc,xgb_acc]
models = ['Logistic Regression', 'Decision Tree Classifier', 'Linear SVC', 'SVM', 'Random Forest Classifier', 'SGDClassifier', 'KNN Classifier','XGboost Classifier']

model_accuracy = {}
model_accuracy['Model'] = models
model_accuracy['Scores'] = acc_list

accuracy_df = pd.DataFrame.from_dict(model_accuracy)
accuracy_df.head()


In [ ]:
plt.figure(figsize = (10,7))
ax = sns.barplot(x = 'Model', y = 'Scores', data = accuracy_df)
plt.xticks(rotation='vertical')
plt.ylim([0,1])
plt.title("Accuracy Scores")

for p in ax.patches:
    ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
             ha='center', va='center', rotation=0, xytext=(0, 20), textcoords='offset points')  #horizontal bars



